In [1]:
import rasterio
import numpy as np

# ---- Input ----
VIIRS_INPUT = r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc"
OUT_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif"

# ---- Define time period (matching ERA5 bands) ----
n_bands = 17544   # hourly for 2015–2016
with rasterio.open(VIIRS_INPUT) as src:
    height, width = src.height, src.width
    src_data = src.read(1)  # read single band
    profile = src.profile
    profile.update(
        count=n_bands,
        dtype="uint8",
        driver="GTiff"
    )

    with rasterio.open(OUT_FILE, "w", **profile) as dst:
        for i in range(n_bands):
            # If VIIRS is half-hourly, we can repeat or interpolate
            # Here, simply repeat the same snapshot for all bands
            dst.write(src_data.astype("uint8"), i + 1)

print(f"✅ Created VIIRS time stack with {n_bands} bands → {OUT_FILE}")


✅ Created VIIRS time stack with 17544 bands → C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif


In [3]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# --- Input files ---
VIIRS_STACK = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif"
DEM_30M = r"C:\Users\Ankit\Datasets_Forest_fire\merged_DEM_30m_32644_aligned_filled.tif"
OUT_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack_30m.tif"

# --- Open DEM as reference grid ---
with rasterio.open(DEM_30M) as dem:
    ref_transform = dem.transform
    ref_crs = dem.crs
    ref_height = dem.height
    ref_width = dem.width

# --- Open VIIRS stack ---
with rasterio.open(VIIRS_STACK) as src:
    profile = src.profile.copy()
    profile.update(
        driver="GTiff",
        height=ref_height,
        width=ref_width,
        crs=ref_crs,
        transform=ref_transform,
        count=src.count,
        dtype="uint8"
    )

    with rasterio.open(OUT_FILE, "w", **profile) as dst:
        for i in range(1, src.count + 1):
            band = src.read(i)

            dst_band = dst.read(1) * 0  # empty target
            reproject(
                source=band,
                destination=dst_band,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.nearest  # preserve binary 0/1
            )
            dst.write(dst_band.astype("uint8"), i)

print(f"✅ VIIRS stack resampled to 30m → {OUT_FILE}")

RasterioIOError: VIIRS_fire_time_stack_30m.tif: Free disk space available is 350308126720 bytes, whereas 2841110728704 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.